In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import scipy.ndimage

# --- CONFIG ---
MODEL_PATH = "/content/drive/MyDrive/NeuroOnco/Modello/cnn_model_with_series.pt"
PAZIENTE_PT = "/content/drive/MyDrive/NeuroOnco/Derivate/MIRRI.pt"

# --- CARICA MODELLO ---
checkpoint = torch.load(MODEL_PATH, map_location=device)

selected_series = checkpoint["selected_series"]
PATCH_SIZE = checkpoint.get("patch_size", (15, 15, 15))  # ✅ Fallback se mancante

print(f"✅ Modello caricato")
print(f"🧠 Serie usate: {selected_series}")
print(f"📐 Dimensione patch dal modello: {PATCH_SIZE}")

model = CNN3D(in_channels=len(selected_series)).to(device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

BATCH_SIZE = 1024
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- STEP per sliding window ---
step_z, step_y, step_x = 7, 7, 7  # passo fine per risoluzione alta

# --- MODELLO ORIGINALE ---
class CNN3D(torch.nn.Module):
    def __init__(self, in_channels, n_classes=2):
        super().__init__()
        self.block1 = torch.nn.Sequential(
            torch.nn.Conv3d(in_channels, 32, 3, padding=1),
            torch.nn.BatchNorm3d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool3d(2),
            torch.nn.Dropout3d(0.2),
        )
        self.block2 = torch.nn.Sequential(
            torch.nn.Conv3d(32, 64, 3, padding=1),
            torch.nn.BatchNorm3d(64),
            torch.nn.ReLU(),
            torch.nn.MaxPool3d(2),
            torch.nn.Dropout3d(0.3),
        )
        self.block3 = torch.nn.Sequential(
            torch.nn.Conv3d(64, 128, 3, padding=1),
            torch.nn.BatchNorm3d(128),
            torch.nn.ReLU(),
            torch.nn.AdaptiveAvgPool3d(1),
        )
        self.fc = torch.nn.Linear(128, n_classes)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

# --- CARICA MODELLO ---
checkpoint = torch.load(MODEL_PATH, map_location=device)
selected_series = checkpoint["selected_series"]
model = CNN3D(in_channels=len(selected_series)).to(device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()
print("✅ Modello caricato (CNN3D)")

# --- CARICA VOLUME ---
pt = torch.load(PAZIENTE_PT)
volume_all = pt["volume"].float()
serie_names = pt["nomi_serie"]
serie_names_lower = [s.lower() for s in serie_names]
selected_indices = [serie_names_lower.index(s.lower()) for s in selected_series]

# 🔵 Volume lasciato su CPU per slicing più veloce
volume = volume_all[selected_indices]  # (C, Z, Y, X)

# --- SLIDING WINDOW MANUALE ---
C, Z, Y, X = volume.shape
pz, py, px = PATCH_SIZE
sz, sy, sx = step_z, step_y, step_x

prob_map = torch.zeros((Z, Y, X), dtype=torch.float32).to("cpu")
weight_map = torch.zeros_like(prob_map)

patches = []
coords = []

print("📦 Inizio generazione patch...")
for z in tqdm(range(0, Z - pz + 1, sz), desc="Z-loop"):
    for y in range(0, Y - py + 1, sy):
        for x in range(0, X - px + 1, sx):
            patch = volume[:, z:z+pz, y:y+py, x:x+px]
            if not torch.isnan(patch).any():
                patches.append(patch)
                coords.append((z, y, x))

print(f"📦 Patch da processare: {len(patches)}")

# --- INFERENZA ---
with torch.no_grad():
    for i in tqdm(range(0, len(patches), BATCH_SIZE), desc="🧠 Inference"):
        batch = patches[i:i+BATCH_SIZE]
        batch_tensor = torch.stack(batch).to(device)  # (B, C, D, H, W)
        logits = model(batch_tensor)
        probs = F.softmax(logits, dim=1)[:, 1].cpu()  # classe MALATO

        for j, prob in enumerate(probs):
            z, y, x = coords[i+j]
            prob_map[z:z+pz, y:y+py, x:x+px] += prob.item()
            weight_map[z:z+pz, y:y+py, x:x+px] += 1

# --- Normalizzazione ---
prob_map /= (weight_map + 1e-8)

# --- Converte in NumPy per post-processing ---
prob_map_np = prob_map.numpy()

# --- Salvataggio della mappa grezza ---
out_path = PAZIENTE_PT.replace(".pt", "_prob_map.npy")
np.save(out_path, prob_map_np)
print("✅ Mappa di probabilità salvata:", out_path)

# --- Filtro Gaussiano per lisciare ---
prob_map_smooth = scipy.ndimage.gaussian_filter(prob_map_np, sigma=1.5)

# --- Salvataggio della mappa liscia ---
out_path_smooth = PAZIENTE_PT.replace(".pt", "_prob_map.npy")
np.save(out_path_smooth, prob_map_smooth)
print("🎨 Mappa salvata:", out_path_smooth)

# --- Info ---
print("📐 Shape mappa:", prob_map_smooth.shape)


✅ Modello caricato
🧠 Serie usate: ['t1w', 'flair', 'dwi', 'ktran', 'cbv', 'apt']
📐 Dimensione patch dal modello: (15, 15, 15)
✅ Modello caricato (CNN3D)
📦 Inizio generazione patch...


Z-loop:  45%|████▍     | 21/47 [00:06<00:08,  3.16it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-fbdc75436c24>", line None, in <cell line: 0>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
          ^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
#DEBUG
import torch
import torch.nn.functional as F
import numpy as np

# CONFIG
MODEL_PATH = "/content/drive/MyDrive/NeuroOnco/Modello/cnn_model_with_series.pt"
PAZIENTE_PT = "/content/drive/MyDrive/NeuroOnco/Derivate/CORZANI1.pt"
PATCH_SIZE = (15, 15, 15)
NUM_PATCHES = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ MODELLO usato nel training
class CNN3D(torch.nn.Module):
    def __init__(self, in_channels, n_classes=2):
        super().__init__()
        self.block1 = torch.nn.Sequential(
            torch.nn.Conv3d(in_channels, 32, 3, padding=1),
            torch.nn.BatchNorm3d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool3d(2),
            torch.nn.Dropout3d(0.2),
        )
        self.block2 = torch.nn.Sequential(
            torch.nn.Conv3d(32, 64, 3, padding=1),
            torch.nn.BatchNorm3d(64),
            torch.nn.ReLU(),
            torch.nn.MaxPool3d(2),
            torch.nn.Dropout3d(0.3),
        )
        self.block3 = torch.nn.Sequential(
            torch.nn.Conv3d(64, 128, 3, padding=1),
            torch.nn.BatchNorm3d(128),
            torch.nn.ReLU(),
            torch.nn.AdaptiveAvgPool3d(1),
        )
        self.fc = torch.nn.Linear(128, n_classes)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

# ✅ Carica modello
checkpoint = torch.load(MODEL_PATH, map_location=device)
selected_series = checkpoint["selected_series"]
model = CNN3D(in_channels=len(selected_series)).to(device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

# ✅ Carica paziente e volume
pt = torch.load(PAZIENTE_PT)
volume_all = pt["volume"].float()
roi_masks = pt["roi_masks"]
serie_names = pt["nomi_serie"]
serie_names_lower = [s.lower() for s in serie_names]
selected_indices = [serie_names_lower.index(s.lower()) for s in selected_series]
volume = volume_all[selected_indices].to(device)  # (C, Z, Y, X)

# ✅ ROI malato e sano
roi_malato = torch.tensor(roi_masks["MALATO"]).to(device)
roi_sano = torch.tensor(roi_masks["SANO"]).to(device)

def estrai_patch(roi_mask, volume, num, label):
    coords = torch.nonzero(roi_mask)
    np.random.seed(0)
    selected = coords[np.random.choice(len(coords), size=num, replace=False)]
    pad = [s // 2 for s in PATCH_SIZE]
    patches = []
    for (z, y, x) in selected:
        z0, z1 = z - pad[0], z + pad[0] + 1
        y0, y1 = y - pad[1], y + pad[1] + 1
        x0, x1 = x - pad[2], x + pad[2] + 1
        if z0 < 0 or y0 < 0 or x0 < 0 or z1 > volume.shape[1] or y1 > volume.shape[2] or x1 > volume.shape[3]:
            continue
        patch = volume[:, z0:z1, y0:y1, x0:x1]
        patches.append(patch)
    return torch.stack(patches), selected

# ✅ Estrai patch
patches_malato, _ = estrai_patch(roi_malato, volume, NUM_PATCHES, "MALATO")
patches_sano, _ = estrai_patch(roi_sano, volume, NUM_PATCHES, "SANO")

# ✅ Inference
with torch.no_grad():
    probs_malato = F.softmax(model(patches_malato.to(device)), dim=1)[:, 1].cpu().numpy()
    probs_sano = F.softmax(model(patches_sano.to(device)), dim=1)[:, 1].cpu().numpy()

# ✅ RISULTATI
print("\n🎯 Probabilità di classe MALATO su patch nella ROI MALATO:")
for i, p in enumerate(probs_malato):
    print(f"Patch {i+1:02d} → {p:.4f}")

print("\n🎯 Probabilità di classe MALATO su patch nella ROI SANO:")
for i, p in enumerate(probs_sano):
    print(f"Patch {i+1:02d} → {p:.4f}")

